In [1]:
import numpy as np
import pandas as pd
from CoolProp.CoolProp import PropsSI, PhaseSI

In [2]:


def get_sat_pressure(T_hot, work_fluid="Water"):
    T_hot += 273.15
    P_sat = PropsSI('P', 'T', T_hot, 'Q', 0, work_fluid)
    return P_sat/1e5


def simulate_rankine(T_hot, T_cold, P_high_bar, work_fluid="Water"):
    # Convert units
    T_hot += 273.15
    T_cold += 273.15
    P_high = P_high_bar * 1e5
    
    # 1: Boiler Outlet
    T_1 = T_hot
    P_1 = P_high
    H_1 = PropsSI('H', 'T', T_1, 'P', P_1, work_fluid)
    S_1 = PropsSI('S', 'T', T_1, 'P', P_1, work_fluid)
    phase_1  = PhaseSI('T', T_1, 'P', P_1, work_fluid)

    # 2: Turbine Outlet
    P_2 = PropsSI('P', 'T', T_cold, 'Q', 0, work_fluid) # we want saturation pressure at outlet
    S_2 = S_1
    H_2 = PropsSI('H', 'P', P_2, 'S', S_2, work_fluid)
    T_2 = PropsSI('T', 'P', P_2, 'S', S_2, work_fluid)
    X_2 = PropsSI('Q', 'P', P_2, 'S', S_2, work_fluid)
    phase_2  = PhaseSI('P', P_2, 'S', S_2, work_fluid)

    # 3: Condenser Outlet
    T_3 = T_cold
    X_3 = 0 # force a saturated liquid
    H_3 = PropsSI('H', 'T', T_3, 'Q', X_3, work_fluid)
    P_3 = PropsSI('P', 'T', T_3, 'Q', X_3, work_fluid)
    S_3 = PropsSI('S', 'T', T_3, 'Q', X_3, work_fluid)
    phase_3  = PhaseSI('T', T_3, 'Q', X_3, work_fluid)

    # 4: Pump outlet
    P_4 = P_high
    S_4 = S_3
    H_4 = PropsSI('H', 'P', P_4, 'S', S_4, work_fluid)
    T_4 = PropsSI('T', 'P', P_4, 'S', S_4, work_fluid)
    phase_4  = PhaseSI('P', P_4, 'S', S_4, work_fluid)

    # Create DataFrame
    states = [
        ['1', phase_1, P_1/1e5, T_1-273.15, '',  H_1/1000, S_1/1000],
        ['2', phase_2, P_2/1e5, T_2-273.15, X_2 if 0 <= X_2 <= 1 else '', H_2/1000, S_2/1000],
        ['3', phase_3, P_3/1e5, T_3-273.15, X_3, H_3/1000, S_3/1000],
        ['4', phase_4, P_4/1e5, T_4-273.15, '',  H_4/1000, S_4/1000],
    ]
    
    df = pd.DataFrame(states, columns=[
        'State', 'Phase', 'Pressure (Bar)', 'Temperature (C)', 'Quality', 
        'Enthalpy (kJ/kg)', 'Entropy (kJ/kg-K)'
    ])

    eta = (H_1 - H_2) / (H_1 - H_4)
    
    return df, eta



# test case with water:
T_hot = 200
T_cold = 25
fluid = "Water"

P_sat = get_sat_pressure(T_hot, fluid)
df, eta = simulate_rankine(T_hot,T_cold, 10, fluid)

print(f"{eta:.4f}")
df


0.3073


,State,Phase,Pressure (Bar),Temperature (C),Quality,Enthalpy (kJ/kg),Entropy (kJ/kg-K)
0,1,gas,10.000000,200.000000,,2828.264476,6.695543
1,2,twophase,0.031699,25.000000,0.772743,1991.617177,6.695543
2,3,twophase,0.031699,25.000000,0,104.829222,0.367225
3,4,liquid,10.000000,25.018382,,105.828826,0.367225


In [3]:
# this is the exact design table and efficiency we got from Aspen, so I am fairly sure this is working great!

In [4]:

def optimize_rankine_cycle(work_fluid):
    best_eta = -1
    best_params = None
    best_df = None

    # Search bounds from low temp to high temp
    T_cold_range = np.linspace(25, 200, 50)
    T_hot_range = np.linspace(25, 200, 50) 

    for T_cold in T_cold_range:
        for T_hot in T_hot_range:

            # there is a better way of doing this!
            if T_hot <= T_cold:
                continue 

            try:
                
                P_sat = get_sat_pressure(T_hot, work_fluid)

                # Try a range of values from 0.1 bar to rrrigghhttt under P_sat to avoid coolprop sadness
                for P_high_bar in np.linspace(0.1, 0.999 * P_sat, 20):
                    try:
                        df, eta = simulate_rankine(T_hot, T_cold, P_high_bar, work_fluid)

                        # check if we are in two-phase region -- while possible to have a gas coming out of turbine, this fits the scope of the assignment better
                        phase_2 = df.loc[df['State'] == '2', 'Phase'].values[0]
                        if phase_2 == 'twophase' and eta > best_eta:
                            best_eta = eta
                            best_params = (T_cold, T_hot, P_high_bar)
                            best_df = df
                    except:
                        continue  # coolprop throws lots of helpful errors when performing transitions that don't exist
            except:
                continue

    return best_eta, best_params, best_df


In [5]:
print("Water:")
best_eta, best_params, best_df = optimize_rankine_cycle("Water")
print(f"Best eta: {best_eta:.4f}")
print(f"Best Params: {best_params}")
best_df

Water:
Best eta: 0.3274
Best Params: (25.0, 200.0, 15.53372972568236)


,State,Phase,Pressure (Bar),Temperature (C),Quality,Enthalpy (kJ/kg),Entropy (kJ/kg-K)
0,1,gas,15.533730,200.000000,,2792.121161,6.430814
1,2,twophase,0.031699,25.000000,0.740417,1912.688407,6.430814
2,3,twophase,0.031699,25.000000,0,104.829222,0.367225
3,4,liquid,15.533730,25.028629,,106.383546,0.367225


In [6]:
print("Toluene:")
best_eta, best_params, best_df = optimize_rankine_cycle("Toluene")
print(f"Best eta: {best_eta:.4f}")
print(f"Best Params: {best_params}")
best_df

Toluene:
Best eta: 0.1517
Best Params: (25.0, 85.71428571428572, 0.4712414170334203)


,State,Phase,Pressure (Bar),Temperature (C),Quality,Enthalpy (kJ/kg),Entropy (kJ/kg-K)
0,1,gas,0.471241,85.714286,,327.839865,0.918354
1,2,twophase,0.037993,25.000000,0.998749,254.096407,0.918354
2,3,twophase,0.037993,25.000000,0,-158.239630,-0.464627
3,4,liquid,0.471241,25.009523,,-158.189379,-0.464627


In [7]:
print("R245fa:")
best_eta, best_params, best_df = optimize_rankine_cycle("R245fa")
print(f"Best eta: {best_eta:.4f}")
print(f"Best Params: {best_params}")
best_df

R245fa:
Best eta: 0.2114
Best Params: (25.0, 153.57142857142858, 36.281980189223596)


,State,Phase,Pressure (Bar),Temperature (C),Quality,Enthalpy (kJ/kg),Entropy (kJ/kg-K)
0,1,gas,36.281980,153.571429,,473.609254,1.753795
1,2,twophase,1.485811,25.000000,0.995183,423.286496,1.753795
2,3,twophase,1.485811,25.000000,0,232.982608,1.115513
3,4,liquid,36.281980,26.173984,,235.574950,1.115513


In [8]:
print("R1233zdE:")
best_eta, best_params, best_df = optimize_rankine_cycle("R1233zdE")
print(f"Best eta: {best_eta:.4f}")
print(f"Best Params: {best_params}")
best_df

R1233zdE:
Best eta: 0.2270
Best Params: (25.0, 164.2857142857143, 34.89524235419753)


,State,Phase,Pressure (Bar),Temperature (C),Quality,Enthalpy (kJ/kg),Entropy (kJ/kg-K)
0,1,gas,34.895242,164.285714,,506.730792,1.867932
1,2,twophase,1.298069,25.000000,0.994878,451.674186,1.867932
2,3,twophase,1.298069,25.000000,0,261.498598,1.230080
3,4,liquid,34.895242,26.230439,,264.151831,1.230080


In [9]:
print("Cyclopentane:")
best_eta, best_params, best_df = optimize_rankine_cycle("Cyclopentane")
print(f"Best eta: {best_eta:.4f}")
print(f"Best Params: {best_params}")
best_df

Cyclopentane:
Best eta: 0.1373
Best Params: (25.0, 78.57142857142858, 2.423088636177762)


,State,Phase,Pressure (Bar),Temperature (C),Quality,Enthalpy (kJ/kg),Entropy (kJ/kg-K)
0,1,gas,2.423089,78.571429,,425.401250,1.216319
1,2,twophase,0.423444,25.000000,0.998932,360.795824,1.216319
2,3,twophase,0.423444,25.000000,0,-45.433874,-0.146182
3,4,liquid,2.423089,25.059465,,-45.163825,-0.146182


In [10]:


print("n-Pentane")
best_eta, best_params, best_df = optimize_rankine_cycle("nPentane")
print(f"Best eta: {best_eta:.4f}")
print(f"Best Params: {best_params}")
best_df



n-Pentane
Best eta: -1.0000
Best Params: None


In [11]:

print("Ammonia")
best_eta, best_params, best_df = optimize_rankine_cycle("Ammonia")
print(f"Best eta: {best_eta:.4f}")
print(f"Best Params: {best_params}")
best_df


Ammonia
Best eta: 0.2158
Best Params: (25.0, 132.14285714285717, 107.04179499050417)


,State,Phase,Pressure (Bar),Temperature (C),Quality,Enthalpy (kJ/kg),Entropy (kJ/kg-K)
0,1,gas,107.041795,132.142857,,1490.149752,4.604457
1,2,twophase,10.026950,25.000000,0.693755,1271.965802,4.604457
2,3,twophase,10.026950,25.000000,0,463.174878,1.891759
3,4,liquid,107.041795,27.479075,,479.194443,1.891759


In [12]:
print("isobutane")
best_eta, best_params, best_df = optimize_rankine_cycle("isobutane")
print(f"Best eta: {best_eta:.4f}")
print(f"Best Params: {best_params}")
best_df

isobutane
Best eta: 0.1630
Best Params: (46.42857142857143, 132.14285714285717, 34.76290918288151)


,State,Phase,Pressure (Bar),Temperature (C),Quality,Enthalpy (kJ/kg),Entropy (kJ/kg-K)
0,1,gas,34.762909,132.142857,,673.449000,2.325179
1,2,twophase,6.266775,46.428571,0.997776,615.527468,2.325179
2,3,twophase,6.266775,46.428571,0,312.763337,1.377793
3,4,liquid,34.762909,48.208168,,318.189142,1.377793
